<a href="https://colab.research.google.com/github/oijiin/ANALISE_DE_DADOS/blob/main/Simulador_ERP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
from decimal import Decimal # Usar Decimal para valores monetários evita problemas de ponto flutuante

# Reutilizar a classe Endereco do TMS para consistência (ou definir uma específica se necessário)
# from tms_simulator import Endereco # Supondo que a classe Endereco está em tms_simulator.py

In [2]:
# --- Definição de Endereco (copiada do TMS para exemplo autônomo) ---
class Endereco:
    """Representa um endereço geográfico simples."""
    def __init__(self, logradouro, numero, cidade, estado, cep, pais="Brasil"):
        self.logradouro = logradouro
        self.numero = numero
        self.cidade = cidade
        self.estado = estado
        self.cep = cep
        self.pais = pais
    def __str__(self):
        return f"{self.logradouro}, {self.numero} - {self.cidade}/{self.estado} - CEP: {self.cep}"
    def __repr__(self):
        return f"Endereco(logradouro='{self.logradouro}', ..., cep='{self.cep}')"
# --- Fim da Definição de Endereco ---

In [3]:
class ProdutoERP:
    """Representa um produto no ERP, com dados comerciais e de custo."""
    def __init__(self, sku, nome, unidade_medida="UN", preco_venda: Decimal = Decimal('0.00'), custo_medio: Decimal = Decimal('0.00')):
        if not sku or not nome:
            raise ValueError("SKU e Nome do produto são obrigatórios.")
        if preco_venda < 0 or custo_medio < 0:
             raise ValueError("Preço de venda e custo médio não podem ser negativos.")

        self.sku = sku
        self.nome = nome
        self.unidade_medida = unidade_medida
        self.preco_venda = preco_venda
        self.custo_medio = custo_medio # Pode ser atualizado por recebimentos
        self.quantidade_em_estoque = 0 # Quantidade total (sincronizada com WMS)
        self.valor_total_estoque = Decimal('0.00') # Calculado: quantidade * custo_medio

    def atualizar_estoque(self, nova_quantidade, novo_custo_medio=None):
        """Atualiza a quantidade e opcionalmente o custo médio."""
        # Idealmente, o custo médio seria recalculado com base nas entradas (FIFO, Média Ponderada)
        # Simplificação: Aceita um novo custo médio ou mantém o antigo
        self.quantidade_em_estoque = nova_quantidade
        if novo_custo_medio is not None and novo_custo_medio >= 0:
            self.custo_medio = novo_custo_medio
        self._recalcular_valor_estoque()
        print(f"INFO ERP Estoque: SKU {self.sku} - Qtd: {self.quantidade_em_estoque}, Custo Médio: {self.custo_medio:.2f}, Valor Total: {self.valor_total_estoque:.2f}")

    def _recalcular_valor_estoque(self):
        """Recalcula o valor total do estoque para este produto."""
        self.valor_total_estoque = Decimal(self.quantidade_em_estoque) * self.custo_medio

    def __str__(self):
        return f"Produto(SKU: {self.sku}, Nome: {self.nome}, Preço Venda: R$ {self.preco_venda:.2f}, Custo Médio: R$ {self.custo_medio:.2f}, Estoque: {self.quantidade_em_estoque} {self.unidade_medida})"

In [4]:
class ParceiroNegocio:
    """Classe base para Clientes e Fornecedores."""
    def __init__(self, id_parceiro, nome, cnpj_cpf, endereco: Endereco):
        if not id_parceiro or not nome or not cnpj_cpf:
            raise ValueError("ID, Nome e CNPJ/CPF são obrigatórios para Parceiros de Negócio.")
        self.id_parceiro = id_parceiro
        self.nome = nome
        self.cnpj_cpf = cnpj_cpf # Validação de formato seria ideal
        self.endereco = endereco

    def __str__(self):
         return f"ID: {self.id_parceiro}, Nome: {self.nome}, Doc: {self.cnpj_cpf}, End: {self.endereco.cidade}/{self.endereco.estado}"

In [5]:
class Cliente(ParceiroNegocio):
    """Representa um Cliente."""
    def __init__(self, id_parceiro, nome, cnpj_cpf, endereco: Endereco, limite_credito: Decimal = Decimal('0.00')):
        super().__init__(id_parceiro, nome, cnpj_cpf, endereco)
        self.limite_credito = limite_credito
        # Outros dados: condições de pagamento, vendedor associado, etc.

    def __str__(self):
        return f"Cliente({super().__str__()}, Limite Crédito: R$ {self.limite_credito:.2f})"

In [6]:
class Fornecedor(ParceiroNegocio):
    """Representa um Fornecedor."""
    def __init__(self, id_parceiro, nome, cnpj_cpf, endereco: Endereco, prazo_entrega_medio_dias=10):
        super().__init__(id_parceiro, nome, cnpj_cpf, endereco)
        self.prazo_entrega_medio_dias = prazo_entrega_medio_dias
        # Outros dados: condições de pagamento, itens fornecidos, etc.

    def __str__(self):
        return f"Fornecedor({super().__str__()}, Prazo Médio: {self.prazo_entrega_medio_dias} dias)"

In [7]:
class ItemPedido:
    """Representa uma linha de item em uma Ordem de Venda ou Compra."""
    def __init__(self, produto: ProdutoERP, quantidade: int, preco_unitario: Decimal):
        if quantidade <= 0:
            raise ValueError("Quantidade no item do pedido deve ser positiva.")
        if preco_unitario < 0:
             raise ValueError("Preço unitário não pode ser negativo.")

        self.produto = produto
        self.quantidade = quantidade
        self.preco_unitario = preco_unitario # Preço de venda (OV) ou Custo (OC)
        self.valor_total_item = Decimal(quantidade) * preco_unitario

    def __str__(self):
        return f"Item(SKU: {self.produto.sku}, Qtd: {self.quantidade}, Preço Unit: R$ {self.preco_unitario:.2f}, Total: R$ {self.valor_total_item:.2f})"

In [8]:
class OrdemVenda:
    """Representa uma Ordem de Venda (Pedido do Cliente)."""
    STATUS_ABERTA = "ABERTA"
    STATUS_LIBERADA_WMS = "LIBERADA_WMS" # WMS pode iniciar picking
    STATUS_SEPARACAO_CONCLUIDA = "SEPARACAO_CONCLUIDA" # WMS informa conclusão
    STATUS_ENVIADA = "ENVIADA" # TMS informa envio
    STATUS_ENTREGUE = "ENTREGUE" # TMS informa entrega
    STATUS_FATURADA = "FATURADA" # Financeiro simula faturamento
    STATUS_CANCELADA = "CANCELADA"

    def __init__(self, id_ov, cliente: Cliente, endereco_entrega: Endereco, itens: list[ItemPedido]):
        self.id_ov = id_ov
        self.cliente = cliente
        self.endereco_entrega = endereco_entrega
        self.itens = itens
        self.data_criacao = datetime.datetime.now()
        self.status = OrdemVenda.STATUS_ABERTA
        self.valor_total_ov = sum(item.valor_total_item for item in itens)
        self.id_carga_tms = None # Para vincular à carga do TMS
        self.tracking_history = [(self.data_criacao, self.status, "Ordem de Venda criada.")]

    def atualizar_status(self, novo_status, detalhes, id_carga=None):
        """Atualiza o status da OV e registra no histórico."""
        # Adicionar validação de transição de status se necessário
        self.status = novo_status
        timestamp = datetime.datetime.now()
        self.tracking_history.append((timestamp, self.status, detalhes))
        if id_carga: self.id_carga_tms = id_carga
        print(f"INFO ERP OV {self.id_ov}: Status atualizado para {self.status}. Detalhes: {detalhes}")

    def __str__(self):
        itens_str = "\n    ".join(map(str, self.itens))
        hist_str = "\n    ".join([f"{ts.strftime('%Y-%m-%d %H:%M')} - {st} - {det}" for ts, st, det in self.tracking_history])
        return (f"Ordem Venda(ID: {self.id_ov}, Status: {self.status})\n"
                f"  Cliente: {self.cliente.nome} (ID: {self.cliente.id_parceiro})\n"
                f"  Entrega: {self.endereco_entrega}\n"
                f"  Valor Total: R$ {self.valor_total_ov:.2f}\n"
                f"  Carga TMS: {self.id_carga_tms}\n"
                f"  Itens:\n    {itens_str}\n"
                f"  Histórico:\n    {hist_str}")

In [9]:
class OrdemCompra:
    """Representa uma Ordem de Compra (Pedido ao Fornecedor)."""
    STATUS_ABERTA = "ABERTA"
    STATUS_ENVIADA_FORNECEDOR = "ENVIADA_FORNECEDOR"
    STATUS_RECEBIMENTO_PENDENTE = "RECEBIMENTO_PENDENTE" # Aguardando WMS
    STATUS_RECEBIDA_PARCIAL = "RECEBIDA_PARCIAL"
    STATUS_RECEBIDA_TOTAL = "RECEBIDA_TOTAL" # WMS confirma recebimento
    STATUS_LIQUIDADA = "LIQUIDADA" # Financeiro simula pagamento
    STATUS_CANCELADA = "CANCELADA"

    def __init__(self, id_oc, fornecedor: Fornecedor, endereco_entrega_erp: Endereco, itens: list[ItemPedido]):
         # Endereco de entrega aqui é o do nosso armazém
        self.id_oc = id_oc
        self.fornecedor = fornecedor
        self.endereco_entrega = endereco_entrega_erp # Onde vamos receber
        self.itens = itens # Aqui o preco_unitario é o CUSTO
        self.data_criacao = datetime.datetime.now()
        self.status = OrdemCompra.STATUS_ABERTA
        self.valor_total_oc = sum(item.valor_total_item for item in itens)
        self.tracking_history = [(self.data_criacao, self.status, "Ordem de Compra criada.")]
        self.qtd_recebida_por_item = {item.produto.sku: 0 for item in itens} # Controle de recebimento

    def atualizar_status(self, novo_status, detalhes):
        """Atualiza o status da OC e registra no histórico."""
        self.status = novo_status
        timestamp = datetime.datetime.now()
        self.tracking_history.append((timestamp, self.status, detalhes))
        print(f"INFO ERP OC {self.id_oc}: Status atualizado para {self.status}. Detalhes: {detalhes}")

    def registrar_recebimento_item(self, sku, quantidade_recebida):
        """Atualiza a quantidade recebida de um item (chamado pelo WMS)."""
        if sku not in self.qtd_recebida_por_item:
            print(f"ERRO ERP OC {self.id_oc}: SKU {sku} não pertence a esta Ordem de Compra.")
            return False
        self.qtd_recebida_por_item[sku] += quantidade_recebida

        # Verifica se a OC foi totalmente recebida
        totalmente_recebida = True
        parcialmente_recebida = False
        for item in self.itens:
            if self.qtd_recebida_por_item[item.produto.sku] < item.quantidade:
                totalmente_recebida = False
            if self.qtd_recebida_por_item[item.produto.sku] > 0:
                parcialmente_recebida = True

        novo_status = self.status
        if totalmente_recebida:
            novo_status = OrdemCompra.STATUS_RECEBIDA_TOTAL
        elif parcialmente_recebida:
            novo_status = OrdemCompra.STATUS_RECEBIDA_PARCIAL
        else:
            # Se nada foi recebido ainda (improvável chegar aqui), mantém status anterior
            pass

        if novo_status != self.status:
            detalhes = f"Recebimento atualizado para SKU {sku} (Qtd: {quantidade_recebida}). Novo status: {novo_status}"
            self.atualizar_status(novo_status, detalhes)
        else:
             print(f"INFO ERP OC {self.id_oc}: Recebimento registrado para SKU {sku} (Qtd: {quantidade_recebida}). Status permanece: {self.status}")
        return True

    def __str__(self):
        itens_str = "\n    ".join(map(str, self.itens))
        hist_str = "\n    ".join([f"{ts.strftime('%Y-%m-%d %H:%M')} - {st} - {det}" for ts, st, det in self.tracking_history])
        rec_str = ", ".join([f"{sku}: {qtd_rec}/{[i.quantidade for i in self.itens if i.produto.sku==sku][0]}" for sku, qtd_rec in self.qtd_recebida_por_item.items()])
        return (f"Ordem Compra(ID: {self.id_oc}, Status: {self.status})\n"
                f"  Fornecedor: {self.fornecedor.nome} (ID: {self.fornecedor.id_parceiro})\n"
                f"  Receber em: {self.endereco_entrega}\n"
                f"  Valor Total: R$ {self.valor_total_oc:.2f}\n"
                f"  Recebimento (Recebido/Pedido):\n    {rec_str}\n"
                f"  Itens:\n    {itens_str}\n"
                f"  Histórico:\n    {hist_str}")

In [10]:
class RegistroFinanceiro:
    """Simples registro de um evento financeiro."""
    def __init__(self, tipo, valor: Decimal, conta_debito, conta_credito, referencia="", descricao=""):
        self.timestamp = datetime.datetime.now()
        self.tipo = tipo # Ex: 'RECEITA_VENDA', 'CUSTO_FRETE', 'CUSTO_MERCADORIA_VENDIDA', 'ENTRADA_ESTOQUE_COMPRA', 'PAGTO_FORNECEDOR'
        self.valor = valor
        self.conta_debito = conta_debito # Simulação de contas contábeis
        self.conta_credito = conta_credito
        self.referencia = referencia # Ex: ID da OV, OC, Carga TMS
        self.descricao = descricao

    def __str__(self):
        return (f"{self.timestamp.strftime('%Y-%m-%d %H:%M')} - {self.tipo} - R$ {self.valor:.2f} "
                f"[D: {self.conta_debito}, C: {self.conta_credito}] Ref: {self.referencia} Desc: {self.descricao}")

In [11]:
class ERP:
    """
      Sistema ERP Simulado - Orquestrador Central.
    """
    def __init__(self, nome_empresa):
        """Inicializa o ERP com um nome de empresa."""
        self.nome_empresa = nome_empresa
        self.produtos = {} # {sku: objeto ProdutoERP}
        self.clientes = {} # {id_cliente: objeto Cliente}
        self.fornecedores = {} # {id_fornecedor: objeto Fornecedor}
        self.ordens_venda = {} # {id_ov: objeto OrdemVenda}
        self.ordens_compra = {} # {id_oc: objeto OrdemCompra}
        self.log_financeiro = [] # Lista de Registros Financeiros

    # --- Métodos de Cadastro (Dados Mestres) ---
    def adicionar_produto(self, produto: ProdutoERP):
        """
          Adiciona um produto ao catálogo.
          Retorna True se o produto for adicionado, False caso contrário.
        """
        if produto.sku in self.produtos:
            print(f"AVISO ERP: Produto SKU {produto.sku} já existe.")
            return False
        self.produtos[produto.sku] = produto
        print(f"INFO ERP: Produto {produto.sku} adicionado.")
        return True

    def adicionar_cliente(self, cliente: Cliente):
        """
        Adiciona um cliente ao sistema.
        Retorna True se o cliente for adicionado, False caso contrário.

        Args: cliente (Cliente): O cliente a ser adicionado.
        Returns: bool: True se o cliente for adicionado, False caso contrário.
        """
        if cliente.id_parceiro in self.clientes:
            print(f"AVISO ERP: Cliente ID {cliente.id_parceiro} já existe.")
            return False
        self.clientes[cliente.id_parceiro] = cliente
        print(f"INFO ERP: Cliente {cliente.nome} adicionado.")
        return True

    def adicionar_fornecedor(self, fornecedor: Fornecedor):
        """
        Adiciona um fornecedor ao sistema.

        Args: fornecedor (Fornecedor): O fornecedor a ser adicionado.
        Returns: bool: True se o fornecedor for adicionado, False caso contrário.
        """
        if fornecedor.id_parceiro in self.fornecedores:
            print(f"AVISO ERP: Fornecedor ID {fornecedor.id_parceiro} já existe.")
            return False
        self.fornecedores[fornecedor.id_parceiro] = fornecedor
        print(f"INFO ERP: Fornecedor {fornecedor.nome} adicionado.")
        return True

    def get_produto(self, sku):
        return self.produtos.get(sku)
    def get_cliente(self, id_cliente):
        return self.clientes.get(id_cliente)
    def get_fornecedor(self, id_fornecedor):
        return self.fornecedores.get(id_fornecedor)

    # --- Métodos de Processos ---
    def criar_ordem_venda(self, id_ov, id_cliente, itens_pedido_info: list[tuple[str, int]]):
        """Cria uma Ordem de Venda."""
        if id_ov in self.ordens_venda:
            print(f"ERRO ERP: Ordem de Venda {id_ov} já existe.")
            return None
        cliente = self.get_cliente(id_cliente)
        if not cliente:
            print(f"ERRO ERP: Cliente {id_cliente} não encontrado.")
            return None

        itens_ov = []
        for sku, qtd in itens_pedido_info:
            produto = self.get_produto(sku)
            if not produto:
                print(f"ERRO ERP: Produto {sku} não encontrado no catálogo para OV {id_ov}.")
                return None
            # Aqui deveríamos verificar disponibilidade (ATP - Available to Promise)
            if produto.quantidade_em_estoque < qtd:
                 print(f"AVISO ERP OV {id_ov}: Estoque insuficiente para {sku}. Pedido: {qtd}, Disponível: {produto.quantidade_em_estoque}. Criando mesmo assim (sem ATP).")
                 # Numa versão real, bloquearia ou criaria parcial/backorder
            item = ItemPedido(produto, qtd, produto.preco_venda)
            itens_ov.append(item)

        if not itens_ov:
            print(f"ERRO ERP: Nenhum item válido para criar OV {id_ov}.")
            return None

        nova_ov = OrdemVenda(id_ov, cliente, cliente.endereco, itens_ov)
        self.ordens_venda[id_ov] = nova_ov
        print(f"INFO ERP: Ordem de Venda {id_ov} criada para cliente {cliente.nome}.")
        # Próximo passo: liberar para WMS
        self.liberar_ov_para_wms(id_ov)
        return nova_ov

    def criar_ordem_compra(self, id_oc, id_fornecedor, itens_pedido_info: list[tuple[str, int, Decimal]]):
        """Cria uma Ordem de Compra."""
        if id_oc in self.ordens_compra:
            print(f"ERRO ERP: Ordem de Compra {id_oc} já existe.")
            return None
        fornecedor = self.get_fornecedor(id_fornecedor)
        if not fornecedor:
            print(f"ERRO ERP: Fornecedor {id_fornecedor} não encontrado.")
            return None

        itens_oc = []
        # Endereço de entrega da OC é o do nosso armazém (precisaria ser configurado no ERP)
        endereco_nosso_armazem = Endereco("Rua do Armazém ERP", "100", "São Paulo", "SP", "01000-000") # Exemplo

        for sku, qtd, custo_unitario in itens_pedido_info:
            produto = self.get_produto(sku)
            if not produto:
                # Se o produto não existe, podemos cadastrá-lo ou dar erro
                # print(f"AVISO ERP OC {id_oc}: Produto {sku} não encontrado, adicionando ao catálogo...")
                # produto = ProdutoERP(sku, f"Produto {sku}", custo_medio=custo_unitario)
                # self.adicionar_produto(produto)
                # OU
                print(f"ERRO ERP: Produto {sku} não encontrado no catálogo para OC {id_oc}.")
                return None

            # Aqui podemos usar o custo informado ou buscar um custo padrão do fornecedor
            item = ItemPedido(produto, qtd, custo_unitario)
            itens_oc.append(item)

        if not itens_oc:
             print(f"ERRO ERP: Nenhum item válido para criar OC {id_oc}.")
             return None

        nova_oc = OrdemCompra(id_oc, fornecedor, endereco_nosso_armazem, itens_oc)
        self.ordens_compra[id_oc] = nova_oc
        print(f"INFO ERP: Ordem de Compra {id_oc} criada para fornecedor {fornecedor.nome}.")
        # Próximo passo: enviar para fornecedor (simulado)
        nova_oc.atualizar_status(OrdemCompra.STATUS_ENVIADA_FORNECEDOR, "OC enviada ao fornecedor.")
        nova_oc.atualizar_status(OrdemCompra.STATUS_RECEBIMENTO_PENDENTE, "Aguardando recebimento no WMS.")
        return nova_oc

    # --- Métodos de Integração (Simulados) ---
    def liberar_ov_para_wms(self, id_ov):
        """Simula a liberação da OV para o WMS iniciar o picking."""
        if id_ov in self.ordens_venda:
            ov = self.ordens_venda[id_ov]
            if ov.status == OrdemVenda.STATUS_ABERTA:
                 ov.atualizar_status(OrdemVenda.STATUS_LIBERADA_WMS, "Pedido liberado para separação no WMS.")
                 # Aqui você chamaria uma função no seu objeto WMS:
                 # wms.receber_pedido_separacao(id_ov, ov.itens)
                 print(f"INFO ERP -> WMS: Solicitação de separação para OV {id_ov} enviada.")
            else:
                 print(f"AVISO ERP: OV {id_ov} não está no status ABERTA para liberar para WMS (Status: {ov.status}).")
        else:
            print(f"ERRO ERP: OV {id_ov} não encontrada para liberar.")

    def receber_confirmacao_separacao_wms(self, id_ov, itens_separados: list[tuple[str, int]]):
        """Simula o recebimento da confirmação de picking do WMS."""
        if id_ov in self.ordens_venda:
             ov = self.ordens_venda[id_ov]
             # Aqui podemos verificar se o que foi separado bate com o pedido
             ov.atualizar_status(OrdemVenda.STATUS_SEPARACAO_CONCLUIDA, f"WMS confirmou separação de {len(itens_separados)} SKU(s).")
             # Próximo passo: solicitar transporte ao TMS
             self.solicitar_transporte_tms(id_ov)
        else:
            print(f"ERRO ERP: OV {id_ov} não encontrada para confirmar separação.")

    def solicitar_transporte_tms(self, id_ov):
        """Simula a criação de uma solicitação de carga no TMS."""
        if id_ov in self.ordens_venda:
            ov = self.ordens_venda[id_ov]
            if ov.status == OrdemVenda.STATUS_SEPARACAO_CONCLUIDA:
                # Montar a lista de ItemTransporte para o TMS
                itens_carga = []
                for item_ov in ov.itens:
                    # Precisamos de peso/volume aqui - buscamos no ProdutoERP ou temos q ter no WMS/Item
                    peso_simulado = Decimal('0.5') # Exemplo
                    volume_simulado = Decimal('0.001') # Exemplo
                    item_tms = ItemTransporte(item_ov.produto.sku, item_ov.quantidade, float(peso_simulado), float(volume_simulado)) # TMS usa float por enquanto
                    itens_carga.append(item_tms)

                # Endereço de origem é o do nosso armazém (precisaria ser configurado)
                endereco_origem_wms = Endereco("Rua do Armazém ERP", "100", "São Paulo", "SP", "01000-000") # Exemplo

                print(f"INFO ERP -> TMS: Solicitando criação de carga para OV {id_ov}.")
                # Aqui você chamaria uma função no seu objeto TMS:
                # carga_criada = tms.criar_carga(f"Carga_{id_ov}", endereco_origem_wms, ov.endereco_entrega, itens_carga)
                # if carga_criada:
                #    ov.atualizar_status(ov.status, f"Carga {carga_criada.id_carga} criada no TMS.", id_carga=carga_criada.id_carga)
                #    tms.planejar_transporte(carga_criada.id_carga) # Poderia já disparar o planejamento
                # else:
                #    print(f"ERRO ERP: Falha ao criar carga no TMS para OV {id_ov}")
                # Simulação simplificada: Apenas atualiza status da OV
                ov.atualizar_status(OrdemVenda.STATUS_ENVIADA, "Solicitação de transporte enviada ao TMS (simulado).", id_carga=f"CARGA_{id_ov}")

            else:
                print(f"AVISO ERP: OV {id_ov} não está no status SEPARACAO_CONCLUIDA para solicitar transporte (Status: {ov.status}).")
        else:
            print(f"ERRO ERP: OV {id_ov} não encontrada para solicitar transporte.")

    def receber_confirmacao_entrega_tms(self, id_carga_tms, data_entrega):
        """Simula o recebimento da confirmação de entrega do TMS."""
        # Encontra a OV associada à carga
        ov_encontrada = None
        for ov in self.ordens_venda.values():
            if ov.id_carga_tms == id_carga_tms:
                ov_encontrada = ov
                break

        if ov_encontrada:
            if ov_encontrada.status == OrdemVenda.STATUS_ENVIADA:
                detalhes = f"TMS confirmou entrega da carga {id_carga_tms} em {data_entrega.strftime('%Y-%m-%d')}."
                ov_encontrada.atualizar_status(OrdemVenda.STATUS_ENTREGUE, detalhes)
                # Próximo passo: faturar (simulado)
                self.faturar_ordem_venda(ov_encontrada.id_ov)
            else:
                 print(f"AVISO ERP: OV {ov_encontrada.id_ov} (Carga {id_carga_tms}) não está no status ENVIADA para confirmar entrega (Status: {ov_encontrada.status}).")
        else:
            print(f"ERRO ERP: Nenhuma OV encontrada associada à carga TMS {id_carga_tms}.")

    def faturar_ordem_venda(self, id_ov):
        """Simula o processo de faturamento e registro financeiro."""
        if id_ov in self.ordens_venda:
            ov = self.ordens_venda[id_ov]
            if ov.status == OrdemVenda.STATUS_ENTREGUE:
                ov.atualizar_status(OrdemVenda.STATUS_FATURADA, "Ordem de Venda faturada.")
                # Registrar financeiro: Receita e Contas a Receber
                reg_receita = RegistroFinanceiro("RECEITA_VENDA", ov.valor_total_ov,
                                                 "CONTAS_RECEBER", "RECEITA_VENDAS_PRODUTOS",
                                                 referencia=id_ov, descricao=f"Faturamento OV {id_ov}")
                self.log_financeiro.append(reg_receita)

                # Registrar financeiro: Baixa de estoque (Custo da Mercadoria Vendida - CMV)
                custo_total_ov = sum(item.quantidade * item.produto.custo_medio for item in ov.itens)
                reg_cmv = RegistroFinanceiro("CUSTO_MERCADORIA_VENDIDA", custo_total_ov,
                                             "CMV", "ESTOQUE_PRODUTOS_ACABADOS",
                                             referencia=id_ov, descricao=f"Baixa estoque OV {id_ov}")
                self.log_financeiro.append(reg_cmv)
                print(f"INFO ERP Financeiro: Registros de faturamento e CMV para OV {id_ov} criados.")

            else:
                print(f"AVISO ERP: OV {id_ov} não está no status ENTREGUE para faturar (Status: {ov.status}).")
        else:
             print(f"ERRO ERP: OV {id_ov} não encontrada para faturar.")

    def registrar_custo_frete_tms(self, id_carga_tms, custo_frete: Decimal):
         """Registra o custo do frete informado pelo TMS."""
         # Encontra a OV associada
         ov_ref = ""
         for ov in self.ordens_venda.values():
            if ov.id_carga_tms == id_carga_tms:
                ov_ref = ov.id_ov
                break

         reg_frete = RegistroFinanceiro("CUSTO_FRETE_VENDA", custo_frete,
                                       "DESPESA_FRETES", "CONTAS_PAGAR_TRANSPORTADORAS",
                                       referencia=id_carga_tms, descricao=f"Custo frete carga {id_carga_tms} (OV: {ov_ref})")
         self.log_financeiro.append(reg_frete)
         print(f"INFO ERP Financeiro: Custo de frete R$ {custo_frete:.2f} para carga {id_carga_tms} registrado.")

    def receber_confirmacao_recebimento_wms(self, id_oc, sku, quantidade_recebida, novo_custo_medio=None):
        """Recebe a confirmação de recebimento do WMS, atualiza OC e estoque."""
        if id_oc in self.ordens_compra:
            oc = self.ordens_compra[id_oc]
            produto = self.get_produto(sku)
            if not produto:
                 print(f"ERRO ERP: Produto {sku} não encontrado ao confirmar recebimento da OC {id_oc}.")
                 return

            # 1. Atualiza controle de recebimento na OC
            oc.registrar_recebimento_item(sku, quantidade_recebida)

            # 2. Atualiza estoque físico no ERP (deve coincidir com WMS)
            # Numa integração real, o WMS informaria o saldo total após o recebimento
            # Simulação: Apenas adiciona a quantidade recebida ao saldo ERP
            quantidade_anterior = produto.quantidade_em_estoque
            nova_quantidade_total = quantidade_anterior + quantidade_recebida
            produto.atualizar_estoque(nova_quantidade_total, novo_custo_medio) # Atualiza qtd e talvez custo

            # 3. Registrar financeiro: Entrada no estoque x Contas a Pagar Fornecedor
            item_oc = next((item for item in oc.itens if item.produto.sku == sku), None)
            if item_oc:
                 valor_entrada_estoque = Decimal(quantidade_recebida) * item_oc.preco_unitario # Custo da OC
                 reg_estoque = RegistroFinanceiro("ENTRADA_ESTOQUE_COMPRA", valor_entrada_estoque,
                                                "ESTOQUE_PRODUTOS_ACABADOS", "CONTAS_PAGAR_FORNECEDORES",
                                                referencia=id_oc, descricao=f"Recebimento {quantidade_recebida}x {sku} OC {id_oc}")
                 self.log_financeiro.append(reg_estoque)
                 print(f"INFO ERP Financeiro: Entrada em estoque registrada para {quantidade_recebida}x {sku} (OC {id_oc}).")
            else:
                 print(f"ERRO ERP: Item {sku} não encontrado nos itens da OC {id_oc} para registro financeiro.")

        else:
            print(f"ERRO ERP: OC {id_oc} não encontrada para confirmar recebimento.")

    # --- Métodos de Consulta ---
    def consultar_ov(self, id_ov):
        if id_ov in self.ordens_venda:
             print("\n--- Detalhes da Ordem de Venda (ERP) ---")
             print(self.ordens_venda[id_ov])
             print("----------------------------------------")
        else:
             print(f"ERRO ERP: Ordem de Venda {id_ov} não encontrada.")

    def consultar_oc(self, id_oc):
        if id_oc in self.ordens_compra:
             print("\n--- Detalhes da Ordem de Compra (ERP) ---")
             print(self.ordens_compra[id_oc])
             print("----------------------------------------")
        else:
             print(f"ERRO ERP: Ordem de Compra {id_oc} não encontrada.")

    def exibir_log_financeiro(self):
        print("\n--- Log Financeiro (ERP) ---")
        if not self.log_financeiro:
            print("Nenhum registro financeiro.")
            return
        for log in self.log_financeiro:
            print(log)
        print("----------------------------\n")

    def exibir_estoque_geral(self):
         print("\n--- Estoque Geral (ERP) ---")
         if not self.produtos:
             print("Nenhum produto cadastrado.")
             return
         total_valor_estoque = Decimal('0.00')
         for produto in self.produtos.values():
             print(f"  SKU: {produto.sku}, Nome: {produto.nome}, Qtd: {produto.quantidade_em_estoque}, Custo Médio: {produto.custo_medio:.2f}, Valor Total: R$ {produto.valor_total_estoque:.2f}")
             total_valor_estoque += produto.valor_total_estoque
         print(f"\n  Valor Total Geral do Estoque: R$ {total_valor_estoque:.2f}")
         print("---------------------------\n")

In [12]:
# --- Exemplo de Uso ---
if __name__ == "__main__":
    # 1. Configurar ERP e Dados Mestres
    meu_erp = ERP("Minha Empresa Simulada")
    print(f"ERP '{meu_erp.nome_empresa}' iniciado.")

    # Endereços
    end_cli1 = Endereco("Rua do Cliente Feliz", "123", "Rio de Janeiro", "RJ", "22000-001")
    end_forn1 = Endereco("Avenida Industrial", "1000", "Guarulhos", "SP", "07000-000")
    end_nosso_armazem = Endereco("Rua do Armazém ERP", "100", "São Paulo", "SP", "01000-000") # Precisa ser consistente

    # Produtos
    p1 = ProdutoERP("SKU001", "Parafuso Sextavado M6", preco_venda=Decimal('1.50'), custo_medio=Decimal('0.50'))
    p2 = ProdutoERP("SKU002", "Porca M6", preco_venda=Decimal('0.80'), custo_medio=Decimal('0.20'))
    p3 = ProdutoERP("SKU003", "Arruela Lisa M6", preco_venda=Decimal('0.30'), custo_medio=Decimal('0.05'))
    meu_erp.adicionar_produto(p1)
    meu_erp.adicionar_produto(p2)
    meu_erp.adicionar_produto(p3)

    # Clientes
    cli1 = Cliente("CLI001", "Cliente Exemplo Ltda", "11.222.333/0001-44", end_cli1, limite_credito=Decimal('10000.00'))
    meu_erp.adicionar_cliente(cli1)

    # Fornecedores
    forn1 = Fornecedor("FORN001", "Fornecedor Peças SA", "55.666.777/0001-88", end_forn1, prazo_entrega_medio_dias=7)
    meu_erp.adicionar_fornecedor(forn1)

    meu_erp.exibir_estoque_geral() # Estoque inicial zero

    print("\n--- Iniciando Processo de Compra ---")
    # 2. Criar Ordem de Compra
    itens_oc1 = [
        ("SKU001", 1000, Decimal('0.51')), # Comprando SKU001 a 0.51
        ("SKU002", 1500, Decimal('0.22'))  # Comprando SKU002 a 0.22
    ]
    oc1 = meu_erp.criar_ordem_compra("OC1001", "FORN001", itens_oc1)
    if oc1: meu_erp.consultar_oc("OC1001")

    # 3. Simular Recebimento (WMS -> ERP)
    print("\n--- Simulando Recebimento WMS -> ERP ---")
    # WMS confirma recebimento total da OC1001 (atualiza custo médio se informado)
    meu_erp.receber_confirmacao_recebimento_wms("OC1001", "SKU001", 1000, novo_custo_medio=Decimal('0.51'))
    meu_erp.receber_confirmacao_recebimento_wms("OC1001", "SKU002", 1500, novo_custo_medio=Decimal('0.22'))
    meu_erp.consultar_oc("OC1001")
    meu_erp.exibir_estoque_geral() # Estoque atualizado
    meu_erp.exibir_log_financeiro()

    print("\n--- Iniciando Processo de Venda ---")
    # 4. Criar Ordem de Venda
    itens_ov1 = [
        ("SKU001", 50),
        ("SKU002", 100)
    ]
    ov1 = meu_erp.criar_ordem_venda("OV5001", "CLI001", itens_ov1)
    if ov1: meu_erp.consultar_ov("OV5001")

    # 5. Simular Interações WMS e TMS
    print("\n--- Simulando Confirmações WMS/TMS -> ERP ---")
    # WMS -> ERP: Separação concluída
    meu_erp.recer_confirmacao_separacao_wms("OV5001", [("SKU001", 50), ("SKU002", 100)])
    meu_erp.consultar_ov("OV5001") # Status deve ser SEPARACAO_CONCLUIDA, carga solicitada

    # TMS -> ERP: Entrega Confirmada (Supondo que TMS criou CARGA_OV5001 e custou R$ 75.50)
    carga_tms_id = "CARGA_OV5001" # ID que o TMS teria gerado
    custo_frete_real = Decimal('75.50')
    data_entrega_real = datetime.datetime.now() + datetime.timedelta(days=2) # Simula 2 dias depois

    # Primeiro, registrar o custo do frete vindo do TMS
    meu_erp.registrar_custo_frete_tms(carga_tms_id, custo_frete_real)

    # Depois, confirmar a entrega vinda do TMS
    meu_erp.receber_confirmacao_entrega_tms(carga_tms_id, data_entrega_real)
    meu_erp.consultar_ov("OV5001") # Status deve ser FATURADA

    meu_erp.exibir_log_financeiro() # Deve incluir receita, cmv e custo de frete
    meu_erp.exibir_estoque_geral() # Estoque deve ter diminuído

ERP 'Minha Empresa Simulada' iniciado.
INFO ERP: Produto SKU001 adicionado.
INFO ERP: Produto SKU002 adicionado.
INFO ERP: Produto SKU003 adicionado.
INFO ERP: Cliente Cliente Exemplo Ltda adicionado.
INFO ERP: Fornecedor Fornecedor Peças SA adicionado.

--- Estoque Geral (ERP) ---
  SKU: SKU001, Nome: Parafuso Sextavado M6, Qtd: 0, Custo Médio: 0.50, Valor Total: R$ 0.00
  SKU: SKU002, Nome: Porca M6, Qtd: 0, Custo Médio: 0.20, Valor Total: R$ 0.00
  SKU: SKU003, Nome: Arruela Lisa M6, Qtd: 0, Custo Médio: 0.05, Valor Total: R$ 0.00

  Valor Total Geral do Estoque: R$ 0.00
---------------------------


--- Iniciando Processo de Compra ---
INFO ERP: Ordem de Compra OC1001 criada para fornecedor Fornecedor Peças SA.
INFO ERP OC OC1001: Status atualizado para ENVIADA_FORNECEDOR. Detalhes: OC enviada ao fornecedor.
INFO ERP OC OC1001: Status atualizado para RECEBIMENTO_PENDENTE. Detalhes: Aguardando recebimento no WMS.

--- Detalhes da Ordem de Compra (ERP) ---
Ordem Compra(ID: OC1001, St

AttributeError: 'ERP' object has no attribute 'recer_confirmacao_separacao_wms'